In [1]:
import braindecode
import mne
from scipy.io import loadmat
import numpy as np
import pandas as pd
import glob
import os
import warnings

from sklearn.model_selection import LeaveOneGroupOut
from braindecode.datautil import create_from_mne_epochs
from braindecode.preprocessing import exponential_moving_standardize
from braindecode.models import EEGNetv4
from braindecode import EEGClassifier
from skorch.callbacks import LRScheduler
import torch
from braindecode.util import set_random_seeds



C:\Users\User\AppData\Roaming\Python\Python312\site-packages\braindecode\datautil\__init__.py:24: UserWarning: create_from_mne_epochs has been moved to datasets, please use from braindecode.datasets import create_from_mne_epochs
  warn(


In [2]:
# GPU check
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True

In [3]:

# Reproducibility
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

In [4]:
# Global variables
window_size = 1024   # 2 sec
window_stride = 64   # 125 ms
low_cut_hz = 8.
high_cut_hz = 32.
factor_new = 1e-3
init_block_size = 1000
n_epochs = 25
batch_size = 32

In [5]:
# Suppress warnings
warnings.filterwarnings('ignore')
mne.set_log_level(verbose=False)

In [9]:
data_path = r"C:\Users\User\Documents\GitHub\Frequency-Adaptive-Temporal-Kernel-EEGNet\Data"
training_files = glob.glob(data_path + "/*T.mat")
print("Number of training files:", len(training_files))

Number of training files: 10


In [10]:
def get_mne_epochs(filepath, t_start=2, fs=512):
    mat_data = loadmat(filepath)
    eeg_data = mat_data['RawEEGData'][:, :, fs*t_start:]  # drop first t_start sec
    labels = mat_data['Labels'].ravel() - 1  # 0/1 labels

    ch_names = ['F3','FC3','C3','CP3','P3','FCz','CPz','F4','FC4','C4','CP4','P4']
    info = mne.create_info(ch_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, tmin=-0.5)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None)
    epochs.events[:, 2] = labels
    return epochs

epochs_list_train = [get_mne_epochs(f) for f in training_files]
print("Number of subjects loaded:", len(epochs_list_train))

Number of subjects loaded: 10


In [11]:
window_size = 1024  # samples (2 sec)
window_stride = 64  # samples (125 ms)

windows_datasets_list = []

for epochs in epochs_list_train:
    cropped_epoch = epochs.crop(tmin=0.5, tmax=4.5, include_tmax=False)
    windows_dataset = create_from_mne_epochs(
        [cropped_epoch],
        window_size_samples=window_size,
        window_stride_samples=window_stride,
        drop_last_window=False
    )
    windows_datasets_list.append(windows_dataset)

print("Subjects (datasets):", len(windows_datasets_list))
print("Windows in first subject:", len(windows_datasets_list[0]))


Subjects (datasets): 10
Windows in first subject: 1360


In [12]:
# Get the first dataset for the first subject
first_dataset = windows_datasets_list[0]

# Get the actual EEG data from the first window
first_window = first_dataset.datasets[0].windows  # this is MNE Epochs object
first_window_data = first_window.get_data()        # NumPy array: (trials, channels, samples)

# Number of channels and window size
n_chans = first_window_data.shape[1]
window_size = first_window_data.shape[2]

print("Channels:", n_chans, "Window size:", window_size)

Channels: 12 Window size: 1024


In [14]:
low_cut_hz = 8.
high_cut_hz = 32.
factor_new = 1e-3
init_block_size = 1000

def custom_exp_moving_std_fn(epochs):
    data = epochs.get_data()
    for i in range(len(data)):
        data[i] = exponential_moving_standardize(data[i],
                                                 factor_new=factor_new,
                                                 init_block_size=init_block_size)
    epochs._data = data
    return epochs

for windows_dataset in windows_datasets_list:
    epochs = windows_dataset.datasets[0].windows
    epochs.load_data()   # ✅ forces preload
    epochs.pick_types(eeg=True)
    epochs.filter(l_freq=8., h_freq=32.)
    custom_exp_moving_std_fn(epochs)

In [16]:
from Frequency_Adaptive_model import AdaptiveEEGNet 
# Deterministic training setup 
torch.backends.cudnn.deterministic = True 
torch.backends.cudnn.benchmark = False
# Creating a model
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, EEGNetv4


n_chans = windows_datasets_list[0][0][0].shape[0]      # EEG channels
n_outputs = 2                                          # number of classes
n_times = windows_datasets_list[0][0][0].shape[1]     # time samples per window

model = EEGNetv4(
    n_chans=n_chans,
    n_outputs=n_outputs,
    n_times=n_times,
    final_conv_length='auto'
)

# Send model to GPU
if cuda:
    model.cuda()
# Send model to GPU
if cuda:
    model.cuda()


In [18]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from torch.utils.data import TensorDataset, DataLoader
from braindecode import EEGClassifier
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from copy import deepcopy

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def training_cross_subject_adaptive_val (windows_datasets_list, base_model,
                                                   n_epochs=25, batch_size=32, val_ratio=0.2):
    """
    Cross-subject AdaptiveEEGNet training (no reinit per epoch).
    Tracks best validation kappa for each subject, prints clean summary table.
    """
    n_subjects = len(windows_datasets_list)
    all_val_acc, all_val_kappa = [], []

    print(f"\n{'='*20} CROSS-SUBJECT VALIDATION {'='*20}\n")

    for test_idx in range(n_subjects):
        print(f"\n🧩 Subject {test_idx+1}/{n_subjects}")

        # ----- Prepare data -----
        train_datasets = [windows_datasets_list[i] for i in range(n_subjects) if i != test_idx]
        X_train, y_train = [], []
        for ds in train_datasets:
            for d in ds.datasets:
                X_train.append(d.windows)
                y_train.append(d.y)
        X_train = np.concatenate(X_train, axis=0)
        y_train = np.concatenate(y_train, axis=0)

        # Split into train/validation
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_train, y_train, test_size=val_ratio, stratify=y_train, random_state=42
        )

        train_tensor = TensorDataset(torch.tensor(X_tr, dtype=torch.float32),
                                     torch.tensor(y_tr, dtype=torch.long))
        val_tensor = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                   torch.tensor(y_val, dtype=torch.long))

        # Fresh model copy
        model = deepcopy(base_model)

        # EEG classifier setup
        clf = EEGClassifier(
            model,
            criterion=torch.nn.CrossEntropyLoss,
            optimizer=torch.optim.AdamW,
            optimizer__lr=0.02,
            optimizer__weight_decay=0.0005,
            batch_size=batch_size,
            train_split=predefined_split(val_tensor),
            callbacks=[("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs-1))],
            device=device,
            iterator_train__shuffle=True
        )

        # ---- Train all epochs ----
        clf.fit(train_tensor, y=None, epochs=n_epochs)

        # ---- Evaluate on validation ----
        val_loader = DataLoader(val_tensor, batch_size=batch_size, shuffle=False)
        clf.module_.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for Xb, yb in val_loader:
                out = clf.module_(Xb.to(device))
                val_preds.extend(out.argmax(1).cpu().numpy())
                val_true.extend(yb.numpy())

        val_acc = accuracy_score(val_true, val_preds)
        val_kappa = cohen_kappa_score(val_true, val_preds)

        all_val_acc.append(val_acc)
        all_val_kappa.append(val_kappa)

        print(f"✅ Val Accuracy: {val_acc:.3f} | Val κappa: {val_kappa:.3f}")

    # ---- Final Summary ----
    print(f"\n{'='*60}")
    print("CROSS-SUBJECT VALIDATION SUMMARY")
    print(f"{'-'*60}")
    print(f"Mean Val Accuracy : {np.mean(all_val_acc):.3f} ± {np.std(all_val_acc):.3f}")
    print(f"Mean Val Kappa    : {np.mean(all_val_kappa):.3f} ± {np.std(all_val_kappa):.3f}")
    print(f"{'='*60}\n")


In [19]:
training_cross_subject_adaptive_val(windows_datasets_list, model, n_epochs=25, batch_size=32)


==================== CROSS-SUBJECT VALIDATION ====================


🧩 Subject 1/10
  epoch    train_loss    valid_acc    valid_loss      lr     dur
-------  ------------  -----------  ------------  ------  ------
      1        0.7269       0.5143        0.7034  0.0200  9.9050
      2        0.7005       0.4996        0.6938  0.0199  9.9339
      3        0.6975       0.5008        0.7070  0.0197  9.9866
      4        0.6999       0.5041        0.6992  0.0192  9.8935
      5        0.6957       0.5074        0.6951  0.0187  9.9962
      6        0.6980       0.5155        0.6938  0.0179  9.7364
      7        0.6857       0.5172        0.7718  0.0171  9.0245
      8        0.6778       0.5323        0.7253  0.0161  61.9965
      9        0.6611       0.5282        0.6780  0.0150  67.6997
     10        0.6488       0.5678        0.6722  0.0138  64.6992
     11        0.6207       0.6887        0.6125  0.0126  61.8673
     12        0.5968       0.5462        0.7851  0.0113  54.4510
